In [ ]:
%pip install --upgrade --quiet langchain langchain-community langchain-openai langchain-experimental neo4j wikipedia tiktoken yfiles_jupyter_graphs

In [3]:
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import List
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import TextLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings

try:
    from google.colab import output
    output.enable_custom_widget_manager()
except:
    pass

In [4]:
import os
from dotenv import load_dotenv
if load_dotenv() == False:
    from google.colab import userdata
    os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
    os.environ["NEO4J_URI"] = userdata.get("NEO4J_URI")
    os.environ["NEO4J_USERNAME"] = userdata.get("NEO4J_USERNAME")
    os.environ["NEO4J_PASSWORD"] = userdata.get("NEO4J_PASSWORD")

In [5]:
raw_documents = TextLoader('data/dragonball.txt').load()
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=125)
documents = text_splitter.split_documents(raw_documents)

In [6]:
llm=ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo")
llm_transformer = LLMGraphTransformer(llm=llm)
graph_documents = llm_transformer.convert_to_graph_documents(documents)

In [8]:
graph = Neo4jGraph(url="neo4j://localhost:7687", username="neo4j", password="8uhbnji9")

In [9]:
graph.add_graph_documents(
    graph_documents,
    baseEntityLabel=True,
    include_source=True
)

In [11]:
# directly show the graph resulting from the given Cypher query
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = "neo4j://localhost:7687",
        auth = ("neo4j","8uhbnji9"))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

In [12]:
showGraph()

GraphWidget(layout=Layout(height='800px', width='100%'))